In [ ]:
S!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio
!pip install SpeechRecognition

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (127 kB/s)
Selecting p

In [ ]:
!pip install transformers sentencepiece datasets tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 9.7 MB/s 
     |████████████████████████████████| 1.2 MB 35.6 MB/s 
     |████████████████████████████████| 362 kB 27.0 MB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 6.6 MB 55.0 MB/s 
     |████████████████████████████████| 140 kB 51.4 MB/s 
     |████████████████████████████████| 212 kB 54.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.8 MB/s 
     |████████████████████████████████| 127 kB 68.2 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 63.4 MB/s 
     |████████████████████████████████| 271 kB 66.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalli

In [ ]:
#imports 
import pyaudio
import speech_recognition as s
from datasets import load_dataset
from google.colab import drive
import numpy as np
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

drive.mount('/content/gdrive')
model_save = '/content/gdrive/My Drive/mt5_translation.pt'

Mounted at /content/gdrive


In [ ]:
#data prep
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small") #generic tokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small") #generic model
model = model.cuda()


Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
#parsing example string
example_input_str = '<jp> This is just a test nbuig.'
#example_input_str = 'これは普通のテスト' translated
#tokenizer testing 
input_ids = tokenizer.encode(example_input_str, return_tensors='pt')
print('Input IDs:', input_ids)

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print('Tokens:', tokens)
#this also filters out unrecognized words from the tokenizer.

Input IDs: tensor([[1042, 3889,  669, 1494,  339, 1627,  259,  262, 2978,  259,  272, 1982,
         1315,  260,    1]])
Tokens: ['▁<', 'jp', '>', '▁This', '▁is', '▁just', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>']


In [ ]:
# Source: https://huggingface.co/datasets/alt

#load dataset
dataset = load_dataset('alt')
training_dataset = dataset['train']
testing_dataset = dataset['test']

#language tokens 
LANGTOKEN = {
    'en': '<en>', #english
    'ja': '<jp>', #japanese
    'zh': '<zh>', #chinese
    'id': '<id>', #indonesian
    'th': '<th>', #thai
    'vi': '<vi>'  #vietnamese
    }

special_tokens_dict = {'additional_special_tokens': list(LANGTOKEN.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

maxseqlen = 20

No config specified, defaulting to: alt/alt-parallel


Generating train split:   0%|          | 0/18094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1004 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

Dataset alt downloaded and prepared to /root/.cache/huggingface/datasets/alt/alt-parallel/1.0.0/e784a3f2a9f6bdf277940de6cc9d700eab852896cd94aad4233caf26008da9ed. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
training_dataset[18093]

{'SNT.URLID': '131892',
 'SNT.URLID.SNTID': '6011 ',
 'translation': {'bg': None,
  'en': None,
  'en_tok': None,
  'fil': '"Ginagawa ng mga crew ang katangi-tanging trabaho doon sa lupa, ngunit ang puno\'t dulo ay lumalaban sila para sa kani-kanilang kaligtasan," sinabi ni Dietrich noong Huwebes.',
  'hi': None,
  'id': None,
  'ja': None,
  'khm': None,
  'lo': 'ໄດເອັດທິດ ໄດ້ເວົ້າໃນວັນອັງຄານ ວ່າ "ພວກກຼ້ໄພ ກຳລັງເຮັດວຽກຢ່າງໜ້າເຫຼືອເຊື່ອຢູ່ພາກພື້ນນັ້ນ ແຕ່ພວກດ່ານຫຼັງແມ່ນພວກເຂົາກຳລັງຕໍ່ສູ້ກັບທຸກບາດກ້າວ."',
  'ms': None,
  'my': None,
  'th': '"หน่วยฉุกเฉินทำงานได้อย่างยอดเยี่ยมในพื้นที่ แต่กล่าวโดยสรุปคือพวกเขาสู้เพื่อพื้นที่ทุก ๆ ฟุต" Dietrich กล่าวในวันอังคาร',
  'vi': None,
  'zh': None},
 'url': 'http://en.wikinews.org/wiki/California_wildfires_continue_to_spread'}

In [ ]:

#intake string, add target lang and tokenize it.
def encode_input_str(text, targetlang, tokenizer, seq_len, langtoken=LANGTOKEN):
    targetLangToken = langtoken[targetlang]

    # Tokenize and add special tokens (make into ids)
    input_ids = tokenizer.encode(
        text=targetLangToken + text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=seq_len)

    return input_ids[0]

#expected output encoding ( similar to  top but different input )
def encode_target_str(text, tokenizer, seq_len, lang_token_map=LANGTOKEN):
    token_ids = tokenizer.encode(
        text=text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=seq_len)

    return token_ids[0]

#training function 
def format_translation_data(translations, lang_token_map, tokenizer, seq_len=128):
    # Choose a random 2 languages for in i/o
    langs = list(lang_token_map.keys())
    input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

    # Get the translations for the batch/dataset
    input_text = translations[input_lang]
    target_text = translations[target_lang]

    if input_text is None or target_text is None:
        return None

    input_token_ids = encode_input_str(
        input_text, target_lang, tokenizer, seq_len, lang_token_map)

    target_token_ids = encode_target_str(
        target_text, tokenizer, seq_len, lang_token_map)

    return input_token_ids, target_token_ids

#training function in batches
def transform_batch(batch, lang_token_map, tokenizer):
    inputs = []
    targets = []
    for translation_set in batch['translation']:
        formatted_data = format_translation_data(
            translation_set, lang_token_map, tokenizer, maxseqlen)

        if formatted_data is None:
            continue

        input_ids, target_ids = formatted_data
        inputs.append(input_ids.unsqueeze(0))
        targets.append(target_ids.unsqueeze(0))

    batch_input_ids = torch.cat(inputs).cuda()
    batch_target_ids = torch.cat(targets).cuda()

    return batch_input_ids, batch_target_ids

#data generator
def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=4):
    dataset = dataset.shuffle()
    for i in range(0, len(dataset), batch_size):
        raw_batch = dataset[i:i + batch_size]
        yield transform_batch(raw_batch, lang_token_map, tokenizer)

In [ ]:
#constants
n_epochs = 8 #how many runs
batch_size = 12 
print_freq = 250 #print out avg loss from the training
checkpoint_freq = 1500 
lr = 5e-4
n_batches = int(np.ceil(len(training_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

# Optimizer mt5 model
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

losses = []

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
model_path = '/content/gdrive/My Drive/mt5_translation.pt'

#evaluate the model / find the loss
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, LANGTOKEN,
                                      tokenizer, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)

#actual training for the machine
for epoch_id in range(n_epochs):
    #randomize data order
    data_generator = get_data_generator(training_dataset, LANGTOKEN,
                                        tokenizer, batch_size)

    for batch_id, (input_batch , label_batch) in tqdm_notebook(enumerate(data_generator), total=n_batches): #input token IDs and expected token IDs
        optimizer.zero_grad()

        #forward pass ( labels is for losses ) ( Predicted outputs )
        model_out = model.forward(
            input_ids=input_batch,
            labels=label_batch)

        #calculate loss and update weights
        loss = model_out.loss
        losses.append(loss.item()) # turn into numpy scaler
        loss.backward()
        optimizer.step()
        scheduler.step() #change the learning rate

        if (batch_id + 1) % print_freq == 0:
          avg_loss = np.mean(losses[-print_freq:])
          print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_id+1, batch_id+1, avg_loss, scheduler.get_last_lr()[0]))
      
        if (batch_id + 1) % checkpoint_freq == 0:
          test_loss = eval_model(model, testing_dataset)
          print('Saving model with test loss of {:.3f}'.format(test_loss))

torch.save(model.state_dict(),model_path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1508 [00:00<?, ?it/s]

Epoch: 1 | Step: 250 | Avg. loss: 8.142 | lr: 0.0004945579370395178
Epoch: 1 | Step: 500 | Avg. loss: 4.090 | lr: 0.0004840924313462827
Epoch: 1 | Step: 750 | Avg. loss: 3.683 | lr: 0.00047362692565304753
Epoch: 1 | Step: 1000 | Avg. loss: 3.483 | lr: 0.0004631614199598125
Epoch: 1 | Step: 1250 | Avg. loss: 3.331 | lr: 0.00045269591426657735
Epoch: 1 | Step: 1500 | Avg. loss: 3.227 | lr: 0.0004422304085733423
Saving model with test loss of 3.353


  0%|          | 0/1508 [00:00<?, ?it/s]

Epoch: 2 | Step: 250 | Avg. loss: 3.149 | lr: 0.00043143000669792364
Epoch: 2 | Step: 500 | Avg. loss: 3.087 | lr: 0.0004209645010046886
Epoch: 2 | Step: 750 | Avg. loss: 3.034 | lr: 0.00041049899531145345
Epoch: 2 | Step: 1000 | Avg. loss: 3.011 | lr: 0.00040003348961821836
Epoch: 2 | Step: 1250 | Avg. loss: 2.990 | lr: 0.00038956798392498327
Epoch: 2 | Step: 1500 | Avg. loss: 2.935 | lr: 0.0003791024782317481
Saving model with test loss of 2.878


...

Epoch: 6 | Step: 250 | Avg. loss: 2.516 | lr: 0.00017891828533154723
Epoch: 6 | Step: 500 | Avg. loss: 2.511 | lr: 0.0001684527796383121
Epoch: 6 | Step: 750 | Avg. loss: 2.505 | lr: 0.00015798727394507702
Epoch: 6 | Step: 1000 | Avg. loss: 2.495 | lr: 0.00014752176825184193
Epoch: 6 | Step: 1250 | Avg. loss: 2.518 | lr: 0.00013705626255860684
Epoch: 6 | Step: 1500 | Avg. loss: 2.502 | lr: 0.00012659075686537175
Saving model with test loss of 2.733


  0%|          | 0/1508 [00:00<?, ?it/s]

Epoch: 7 | Step: 250 | Avg. loss: 2.415 | lr: 0.00011579035498995312
Epoch: 7 | Step: 500 | Avg. loss: 2.428 | lr: 0.00010532484929671802
Epoch: 7 | Step: 750 | Avg. loss: 2.447 | lr: 9.485934360348291e-05
Epoch: 7 | Step: 1000 | Avg. loss: 2.387 | lr: 8.439383791024782e-05
Epoch: 7 | Step: 1250 | Avg. loss: 2.411 | lr: 7.392833221701273e-05
Epoch: 7 | Step: 1500 | Avg. loss: 2.418 | lr: 6.346282652377764e-05
Saving model with test loss of 2.551


  0%|          | 0/1508 [00:00<?, ?it/s]

Epoch: 8 | Step: 250 | Avg. loss: 2.359 | lr: 5.266242464835901e-05
Epoch: 8 | Step: 500 | Avg. loss: 2.357 | lr: 4.219691895512391e-05
Epoch: 8 | Step: 750 | Avg. loss: 2.367 | lr: 3.173141326188882e-05
Epoch: 8 | Step: 1000 | Avg. loss: 2.336 | lr: 2.1265907568653717e-05
Epoch: 8 | Step: 1250 | Avg. loss: 2.316 | lr: 1.080040187541862e-05
Epoch: 8 | Step: 1500 | Avg. loss: 2.356 | lr: 3.348961821835231e-07
Saving model with test loss of 2.797


In [ ]:
#input through speech recognition of a audio file, since colab doesnt have direct access to the computer's mic 
import speech_recognition as sr
r = sr.Recognizer()
sample1 = sr.AudioFile('/content/gdrive/My Drive/Comptech-samples/sample1.wav')
sample2 = sr.AudioFile('/content/gdrive/My Drive/Comptech-samples/sample2.wav')
sample3 = sr.AudioFile('/content/gdrive/My Drive/Comptech-samples/sample3.wav')
sample4 = sr.AudioFile('/content/gdrive/My Drive/Comptech-samples/sample4.wav')
samplepick = sample1
with samplepick as source:
  audio = r.record(source)

text = r.recognize_google(audio)
text

'while the men is working I went with the girl'

In [ ]:
#instead run this. actual run for the machine. not perfect since there is loss.
#test_sentence = 'i would like to buy some food, and then go back home'
test_sentence = text
#test_sentence = '量子 チキンスープ グラス ビッグチュングス'
print('Raw input text:', test_sentence)
print('-----------------------------------')
print('by default, it would be translated in to english')
print('Available translations are')
print('1. English')
print('2. Japanese')
print('3. Chinese')
print('4. Indonesian ')
print('5. Thai')
print('6. Vietnamese')
choice = int(input("choice ( input must be a number ) = "))
endlang = 'en'

if(choice == 1):
  endlang='en'
elif (choice == 2):
  endlang = 'ja'
elif (choice == 3):
  endlang = 'zh'
elif (choice == 4):
  endlang = 'id'
elif (choice == 5):
  endlang = 'th'
elif (choice == 6):
  endlang = 'vi'
else:
  print("incorrect input")
print('Raw input text:', test_sentence)

input_ids = encode_input_str(
    text = test_sentence,
    targetlang = endlang,
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    langtoken = LANGTOKEN)
input_ids = input_ids.unsqueeze(0).cuda()
print('-----------------------------------')
print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=4)

print('-----------------------------------')
print('input ids: ', input_ids)
print('output tokens: ', output_tokens)
print('-----------------------------------')

for token_set in output_tokens:
  print(tokenizer.decode(token_set, skip_special_tokens=True))

Raw input text: while the men is working I went with the girl
-----------------------------------
by default, it would be translated in to english
Available translations are
1. English
2. Japanese
3. Chinese
4. Indonesian 
5. Thai
6. Vietnamese
choice ( input must be a number ) = 2
Raw input text: while the men is working I went with the girl
-----------------------------------
Truncated input text: <jp> while the men is working I went with the girl</s><pad><pad><pad><pad><pad><pad>
-----------------------------------
input ids:  tensor([[250101,    259,   4944,    287,    692,    339,  10357,    336,    259,
          14196,    514,    287,  16603,      1,      0,      0,      0,      0,
              0,      0]], device='cuda:0')
output tokens:  tensor([[     0,    259, 115899, 172931,  20172,  94090,    292,  16245,  56509,
          93537, 106903,   5543,    306,      1,      0],
        [     0,    259, 115899, 172931,  20172,  94090,    292,  16245,  56509,
          93537, 10690